In [ ]:
# !pip install -r requirements10.txt (file removed, needed dependeices below)

# torch~=2.2.0
# numpy~=1.24.3
# matplotlib~=3.8.0
# psutil~=5.9.6
# tqdm~=4.66.1
# scipy~=1.12.0
# pandas~=2.2.0
# seaborn~=0.13.0
# networkx~=3.2.1
# scikit-learn~=1.4.0

# **Set the file paths and notes**

<p>Use option = 0 if you want to do training and validation using 2 of the folds (keeping the 3rd fold for final training and testing).</p>

<p>Use option = 1 if you want to do final training (using 2 folds) and testing (with the 3rd fold).</p>

Be sure to update the name of the files - doing 3 cross-validation in this project is done manually with the prepared data to create comparable results with the rest of the projects. See an example below.

Due to the nature of this team's model, early stopping is carried out by manual experimentation during testing and validation for 3CV.


In [ ]:
option = 1

# path for training and validation (without using the 3rd fold). This is an example from the ClusterCV folder
path_lr_data_tr = 'lr_split_AandB_training.csv'
path_hr_data_tr = 'hr_split_AandB_training.csv'
path_lr_data_valid = 'lr_split_AandB_validation.csv'
path_hr_data_valid = 'hr_split_AandB_validation.csv'

# path for final training and testing (using fold A and B for training and C for testing).This is an example from the ClusterCV folder
path_lr_data = 'lr_split_AandB_finaltraining.csv'
path_hr_data = 'hr_split_AandB_finaltraining.csv'
path_lr_data_test = 'lr_clusterC.csv'
path_hr_data_test = 'hr_clusterC.csv'

# path to save the evaluation metrics to - this means that Cluster C will be used for testing
path_eval_matrics = '10-clusterCV-split3.csv'

In [ ]:
import torch

USE_GPU = False
device = 'cuda' if torch.cuda.is_available() and USE_GPU else 'cpu'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GAT(nn.Module):
    """
    A basic implementation of the GAT layer.

    This layer applies an attention mechanism in the graph convolution process,
    allowing the model to focus on different parts of the neighborhood
    of each node.
    """
    def __init__(self, in_features, out_features, activation=None, layer_norm=False, p=0):
        super(GAT, self).__init__()
        # Initialize the weights, bias, and attention parameters as
        # trainable parameters
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.zeros(out_features))
        self.phi = nn.Parameter(torch.FloatTensor(2 * out_features, 1))
        self.activation = activation

        self.layer_norm = layer_norm
        if layer_norm:
            self.norm_layer = nn.LayerNorm(out_features)

        self.dropout_layer = nn.Dropout(p=p) if p > 0.0 else nn.Identity()

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)
        nn.init.xavier_uniform_(self.phi)

    def forward(self, adj, input):
        """
        Forward pass of the GAT layer.

        Parameters:
        input (Tensor): The input features of the nodes.
        adj (Tensor): The adjacency matrix of the graph.

        Returns:
        Tensor: The output features of the nodes after applying the GAT layer.
        """
        ############# Your code here ############
        ## 1. Apply linear transformation and add bias
        H_k = input @ self.weight + self.bias
        ## 2. Compute the attention scores utilizing the previously
        ## established mechanism.
        ## Note: Keep in mind that employing matrix notation can
        ## optimize this process.
        N, D = H_k.shape
        H_k_1 = H_k @ self.phi[:D]
        H_k_2 = H_k @ self.phi[D:]
        S = H_k_1 + H_k_2.transpose(0, 1)
        ## Apply a non-linearity before masking
        S = F.leaky_relu(S)
        ## 3. Compute mask based on adjacency matrix
        mask = (adj + torch.eye(adj.shape[0]).to(adj.device)) == 0
        ## 4. Apply mask to the pre-attention matrix
        S_masked = torch.where(mask, torch.tensor(-9e15).to(adj.device), S)
        ## 5. Compute attention weights using softmax
        S_normalised = F.softmax(S_masked, dim=1)
        ## 6. Aggregate features based on attention weights
        ## Note: name the last line as `h`
        S_normalised = self.dropout_layer(S_normalised)
        h = S_normalised @ H_k
        ## (9-10 lines of code)
        #########################################
        h = self.norm_layer(h) if self.layer_norm else h
        return self.activation(h) if self.activation else h


class GCN(nn.Module):
    """
    A basic implementation of GCN layer.
    It aggregates information from a node's neighbors
    using mean aggregation.
    """
    def __init__(self, in_features, out_features, activation=None, layer_norm=False, p=0):
        super(GCN, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.zeros(out_features))
        self.activation = activation

        self.layer_norm = layer_norm
        if layer_norm:
            self.norm_layer = nn.LayerNorm(out_features)

        self.dropout_layer = nn.Dropout(p=p) if p > 0.0 else nn.Identity()

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)

    def forward(self, adj, x):
        """
        Forward pass of the GCN layer.

        Parameters:
        input (Tensor): The input features of the nodes.
        adj (Tensor): The adjacency matrix of the graph.

        Returns:
        Tensor: The output features of the nodes after applying the GCN layer.
        """
        # apply dropout
        x = self.dropout_layer(x)
        ############# Your code here ############
        ## Note:
        ## 1. Apply the linear transformation
        transformed = torch.matmul(x, self.weight)
        ## 2. Perform the graph convolution operation
        h = torch.matmul(adj, transformed) + self.bias
        ## Note: rename the last line as `output`
        ## (2 lines of code)
        #########################################
        h = self.norm_layer(h) if self.layer_norm else h
        h = self.activation(h) if self.activation else h
        return h


class GraphUnpool(nn.Module):
    '''
    Graph unpool layer
    '''
    def __init__(self):
        super(GraphUnpool, self).__init__()

    def forward(self, A, X, idx):
        new_X = torch.zeros([A.shape[0], X.shape[1]], device=device)
        new_X[idx] = X
        return A, new_X

class GraphPool(nn.Module):
    '''
    Graph polling layer
    '''
    def __init__(self, k, in_dim):
        super(GraphPool, self).__init__()
        self.k = k
        self.proj = nn.Linear(in_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, A, X):
        scores = self.proj(X)
        # scores = torch.abs(scores)
        scores = torch.squeeze(scores)
        scores = self.sigmoid(scores/100)
        num_nodes = A.shape[0]
        values, idx = torch.topk(scores, int(self.k*num_nodes))
        new_X = X[idx, :]
        values = torch.unsqueeze(values, -1)
        new_X = torch.mul(new_X, values)
        A = A[idx, :]
        A = A[:, idx]
        return A, new_X, idx

class GraphUnet(nn.Module):
    '''
    an improved version os Graph U-Net base a GAT
    '''
    def __init__(self, ks, in_dim, out_dim, dim=320):
        super(GraphUnet, self).__init__()
        self.ks = ks
        self.start_gcn = GAT(in_dim, dim, nn.ReLU(True)).to(device)
        self.bottom_gcn = GAT(dim, dim, nn.ReLU(True)).to(device)
        self.end_gcn = GAT(2*dim, out_dim, nn.ReLU(True)).to(device)
        self.down_gcns = []
        self.up_gcns = []
        self.pools = []
        self.unpools = []
        self.l_n = len(ks)
        for i in range(self.l_n):
            self.down_gcns.append(GAT(dim, dim, nn.ReLU(True), p=0.5).to(device))
            self.up_gcns.append(GAT(dim, dim, nn.ReLU(True), p=0.5).to(device))
            self.pools.append(GraphPool(ks[i], dim).to(device))
            self.unpools.append(GraphUnpool().to(device))

    def forward(self, A, X):
        adj_ms = []
        indices_list = []
        down_outs = []
        X = self.start_gcn(A, X)
        start_gcn_outs = X
        org_X = X
        for i in range(self.l_n):

            X = self.down_gcns[i](A, X)
            adj_ms.append(A)
            down_outs.append(X)
            A, X, idx = self.pools[i](A, X)
            indices_list.append(idx)
        X = self.bottom_gcn(A, X)
        for i in range(self.l_n):
            up_idx = self.l_n - i - 1

            A, idx = adj_ms[up_idx], indices_list[up_idx]
            A, X = self.unpools[i](A, X, idx)
            X = self.up_gcns[i](A, X)
            X = X.add(down_outs[up_idx])
        X = torch.cat([X, org_X], 1)
        X = self.end_gcn(A, X)

        return X, start_gcn_outs

In [ ]:
import numpy as np


def weight_variable_glorot(output_dim):
    '''
    glorot weight initialization method
    '''

    input_dim = output_dim
    init_range = np.sqrt(6.0 / (input_dim + output_dim))
    initial = np.random.uniform(-init_range, init_range,
                                (input_dim, output_dim))

    return initial


In [ ]:
import torch
import torch.nn as nn


class GSRLayer(nn.Module):
    '''
    GSRLayers which is used for predicting an HR connectome from the LR connectivity matrix and feature embeddings of the LR connectome
    '''

    def __init__(self, hr_dim):
        super(GSRLayer, self).__init__()

        self.weights = torch.from_numpy(
            weight_variable_glorot(hr_dim)).type(torch.FloatTensor)
        self.weights = torch.nn.Parameter(
            data=self.weights, requires_grad=True)

    def forward(self, A, X):
        with torch.autograd.set_detect_anomaly(True):

            lr = A
            lr_dim = lr.shape[0]
            f = X
            eig_val_lr, U_lr = torch.linalg.eigh(lr, UPLO='U')

            eye_mat = torch.eye(lr_dim).type(torch.FloatTensor).to(A.device)
            s_d = torch.cat((eye_mat, eye_mat), 0)

            a = torch.matmul(self.weights, s_d)
            b = torch.matmul(a, torch.t(U_lr))
            f_d = torch.matmul(b, f)
            f_d = torch.abs(f_d)
            f_d = f_d.fill_diagonal_(1)
            adj = f_d

            X = torch.mm(adj, adj.t())
            X = (X + X.t())/2
            X = X.fill_diagonal_(1)
        return adj, torch.abs(X)


In [ ]:
import torch
import numpy as np
import os
import scipy.io

path = '/dgl/slim_dataset'
roi_str = 'ROI_FC.mat'
def pad_HR_adj(label, split):

    label = np.pad(label, ((split, split), (split, split)), mode="constant")
    np.fill_diagonal(label, 1)
    return label


def normalize_adj_torch(mx):
    rowsum = mx.sum(1)
    r_inv_sqrt = torch.pow(rowsum, -0.5).flatten()
    r_inv_sqrt[torch.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = torch.diag(r_inv_sqrt)
    mx = torch.matmul(mx, r_mat_inv_sqrt)
    mx = torch.transpose(mx, 0, 1)
    mx = torch.matmul(mx, r_mat_inv_sqrt)
    return mx


def unpad(data, split):

    idx_0 = data.shape[0]-split
    idx_1 = data.shape[1]-split
    train = data[split:idx_0, split:idx_1]
    return train


def extract_data(subject, session_str, parcellation_str, subjects_roi):
    folder_path = os.path.join(
        path, str(subject), session_str, parcellation_str)
    roi_data = scipy.io.loadmat(os.path.join(folder_path, roi_str))
    roi = roi_data['r']

    # Replacing NaN values
    col_mean = np.nanmean(roi, axis=0)
    inds = np.where(np.isnan(roi))
    roi[inds] = 1

    # Taking the absolute values of the matrix
    roi = np.absolute(roi, dtype=np.float32)

    if parcellation_str == 'shen_268':
        roi = np.reshape(roi, (1, 268, 268))
    else:
        roi = np.reshape(roi, (1, 160, 160))

    if subject == 25629:
        subjects_roi = roi
    else:
        subjects_roi = np.concatenate((subjects_roi, roi), axis=0)

    return subjects_roi


def load_data(start_value, end_value):

    subjects_label = np.zeros((1, 268, 268))
    subjects_adj = np.zeros((1, 160, 160))

    for subject in range(start_value, end_value):
        subject_path = os.path.join(path, str(subject))

        if 'session_1' in os.listdir(subject_path):

            subjects_label = extract_data(
                subject, 'session_1', 'shen_268', subjects_label)
            subjects_adj = extract_data(
                subject, 'session_1', 'Dosenbach_160', subjects_adj)

    return subjects_adj, subjects_label


def data():
    subjects_adj, subjects_labels = load_data(25629, 25830)
    test_adj_1, test_labels_1 = load_data(25831, 25863)
    test_adj_2, test_labels_2 = load_data(30701, 30757)
    test_adj = np.concatenate((test_adj_1, test_adj_2), axis=0)
    test_labels = np.concatenate((test_labels_1, test_labels_2), axis=0)
    return subjects_adj, subjects_labels, test_adj, test_labels


In [ ]:


class AGSRNet(nn.Module):
    '''
    Original AGSRNet model class from the github repo https://github.com/basiralab/AGSR-Net
    '''

    def __init__(self, ks, args):
        super(AGSRNet, self).__init__()

        self.lr_dim = args.lr_dim
        self.hr_dim = args.hr_dim
        self.hidden_dim = args.hidden_dim
        self.layer = GSRLayer(self.hr_dim).to(device)
        self.net = GraphUnet(ks, self.lr_dim, self.hr_dim).to(device)
        self.gc1 = GCN(self.hr_dim, self.hidden_dim, nn.ReLU(True), True, p=0.5).to(device)
        self.gc2 = GCN(self.hidden_dim, self.hidden_dim, nn.ReLU(True), True, p=0.5).to(device)
        self.gc3 = GCN(self.hidden_dim, self.hr_dim, nn.ReLU(True)).to(device)

    def forward(self, lr, lr_dim, hr_dim):
        with torch.autograd.set_detect_anomaly(True):

            I = torch.eye(self.lr_dim).type(torch.FloatTensor).to(device)
            A = normalize_adj_torch(lr).type(torch.FloatTensor).to(device)

            # Unet
            net_outs, start_gcn_outs = self.net(A, I)

            # GSRLayer
            adj, z = self.layer(A, net_outs)

            # 3 GCN layers
            z = self.gc1(adj, z)
            z = self.gc2(adj, z)
            z = self.gc3(adj, z)

            z = (z + z.t())/2
            z = z.fill_diagonal_(1)

        return torch.abs(z), net_outs, start_gcn_outs, adj


class Discriminator(nn.Module):
    '''
    GCN based discriminators
    '''
    def __init__(self, args):
        super(Discriminator, self).__init__()

        self.in_layer = GCN(args.hr_dim, 2 * args.hr_dim, nn.LeakyReLU(0.2, True)).to(device)
        self.hidden1 = GCN(2 * args.hr_dim, args.hr_dim, nn.LeakyReLU(0.2, True)).to(device)
        self.hidden2 = GCN(args.hr_dim, args.hr_dim // 2, nn.LeakyReLU(0.2, True)).to(device)
        self.out_layer = GCN(args.hr_dim // 2, 1, nn.Sigmoid()).to(device)

    def forward(self, inputs):
        np.random.seed(1)
        torch.manual_seed(1)

        A = normalize_adj_torch(inputs).to(device)
        # A = A + torch.eye(A.shape[0], dtype=torch.float, device=device)

        H = self.in_layer(A, inputs)
        H = self.hidden1(A, H)
        H = self.hidden2(A, H)
        output = self.out_layer(A, H)

        return output

def add_noise_to_adjacency_matrix(adj_matrix, noise_level=0.05):
    # Assume adj_matrix is a numpy array representing your adjacency matrix
    noise = np.random.randn(*adj_matrix.shape) * noise_level
    noisy_adj_matrix = adj_matrix + noise
    # Ensure the noisy adjacency matrix is still symmetric for undirected graphs
    noisy_adj_matrix = np.maximum(noisy_adj_matrix, 0)  # Remove negative values
    noisy_adj_matrix = (noisy_adj_matrix + noisy_adj_matrix.T) / 2  # Make symmetric
    np.fill_diagonal(noisy_adj_matrix, 1)

    return noisy_adj_matrix



In [ ]:
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim
from tqdm import tqdm

def train(model, subjects_adj, subjects_labels, test_lr, test_hr, args):
    optimizerG = optim.Adam(model.parameters(), lr=args.lr, betas=(0.5, 0.999))
    schedulerG = StepLR(optimizerG, step_size=40, gamma=0.8)
    criterion = nn.MSELoss()

    with tqdm(range(args.epochs), desc='Training') as tepoch:
        for epoch in tepoch:
            epoch_loss = []
            epoch_error = []
            with torch.autograd.set_detect_anomaly(True):
                model.train()
                for lr, hr in zip(subjects_adj, subjects_labels):
                    optimizerG.zero_grad()

                    # augmentation through noise injection
                    hr = add_noise_to_adjacency_matrix(hr, 0.1)
                    lr = add_noise_to_adjacency_matrix(lr, 0.1)

                    hr = pad_HR_adj(hr, args.padding)
                    lr = torch.from_numpy(lr).type(torch.FloatTensor).to(device)
                    padded_hr = torch.from_numpy(hr).type(torch.FloatTensor).to(device)

                    eig_val_hr, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')

                    model_outputs, net_outs, start_gcn_outs, layer_outs = model(
                        lr, args.lr_dim, args.hr_dim)

                    recon_loss = criterion(model_outputs, padded_hr)

                    mse_loss = args.lmbda * criterion(net_outs, start_gcn_outs) + criterion(
                        model.layer.weights, U_hr) + recon_loss

                    mse_loss.backward()
                    optimizerG.step()

                    epoch_loss.append(mse_loss.item())
                    epoch_error.append(recon_loss.item())

            schedulerG.step()

            if (epoch % 10 == 0) or (epoch == args.epochs - 1):
                epoch_test_loss = []
                epoch_test_error = []
                model.eval()
                with torch.no_grad():
                    for lr, hr in zip(test_lr, test_hr):
                        hr = pad_HR_adj(hr, args.padding)
                        lr = torch.from_numpy(lr).type(torch.FloatTensor).to(device)
                        padded_hr = torch.from_numpy(hr).type(torch.FloatTensor).to(device)

                        eig_val_hr, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')
                        model_outputs, net_outs, start_gcn_outs, layer_outs = model(
                            lr, args.lr_dim, args.hr_dim)
                        recon_loss = criterion(model_outputs, padded_hr)
                        mse_loss = args.lmbda * criterion(net_outs, start_gcn_outs) + criterion(
                            model.layer.weights, U_hr) + recon_loss

                        epoch_test_loss.append(mse_loss.item())
                        epoch_test_error.append(recon_loss.item())

            tepoch.set_description(f"Epoch {epoch}")
            tepoch.set_postfix(total_loss=np.mean(epoch_loss),
                               mse_error=f'{np.mean(epoch_error) * 100:.2f}%',
                               total_test_loss=np.mean(epoch_test_loss),
                               mse_test_error=f'{np.mean(epoch_test_error) * 100:.2f}%'
                               )
            print()


def test(model, test_adj, args):

    preds_list = []

    for lr in test_adj:
        all_zeros_lr = not np.any(lr)
        if all_zeros_lr == False :
            lr = torch.from_numpy(lr).type(torch.FloatTensor).to(device)
            preds, a, b, c = model(lr, args.lr_dim, args.hr_dim)
            preds = unpad(preds, args.padding).detach().cpu().numpy()
            preds_list.append(preds)

    return np.stack(preds_list)



In [ ]:
import sys

class Args:
    '''
    Hyperparameters for training
    '''
    def __init__(self):
        self.epochs = 91
        self.lr = 0.001
        self.lmbda = 0.1
        self.lr_dim = 160
        self.hr_dim = 320
        self.hidden_dim = 320
        self.padding = 26
        self.mean_dense = 0.
        self.std_dense = 0.01
        self.mean_gaussian = 0.
        self.std_gaussian = 0.1

class AGRNet:
    '''
    the AGR model class which is based on the github repo https://github.com/basiralab/AGSR-Net
    '''
    def __init__(self, args=None):
        # If args is None, use default arguments
        if args is None:
                self.args = Args()
        else:
                self.args = args

        # ks = [0.9, 0.7, 0.6, 0.5]
        ks = [0.9, 0.8, 0.7, 0.6, 0.5, 0.3]
        self.model = AGSRNet(ks, self.args).to(device)

        print(self.model)
        sys.stdout.flush()

    def train(self, lr_vectors, hr_vectors, lr_test, hr_test):
        """
        Inputs:
                # lr_vectors: N*d e.g.(190, 12720) 12720=(160*159)/2
                # hr_vectors: N*d e.g.(190, 35778) 35778=(268*267)/2
                (190, 160, 160)
                (190, 268, 268)
        Output:
                for each epoch, prints out the mean training MSE error
        """

        train(self.model, lr_vectors, hr_vectors, lr_test, hr_test, self.args)
        model_save_path = './agrnet1.pth'
        torch.save(self.model.state_dict(), model_save_path)

    def predict(self, lr_vectors):
        """
        Inputs:
                # lr_vectors: N*d e.g.(190, 12720) 12720=(160*159)/2
                (190, 160, 160)
        Outputs:
                predicted_hr_vectors: (190, 268, 268)
        """
        model_load_path = './agrnet1.pth'
        model_state_dict = torch.load(model_load_path)
        self.model.load_state_dict(model_state_dict)
        self.model.eval()

        predicted_hr_vectors = test(self.model, lr_vectors, self.args)

        return predicted_hr_vectors



In [ ]:
import numpy as np

class MatrixVectorizer:
    """
    A class for transforming between matrices and vector representations.

    This class provides methods to convert a symmetric matrix into a vector (vectorize)
    and to reconstruct the matrix from its vector form (anti_vectorize), focusing on
    vertical (column-based) traversal and handling of elements.
    """

    def __init__(self):
        """
        Initializes the MatrixVectorizer instance.

        The constructor currently does not perform any actions but is included for
        potential future extensions where initialization parameters might be required.
        """
        pass

    @staticmethod
    def vectorize(matrix, include_diagonal=False):
        """
        Converts a matrix into a vector by vertically extracting elements.

        This method traverses the matrix column by column, collecting elements from the
        upper triangle, and optionally includes the diagonal elements immediately below
        the main diagonal based on the include_diagonal flag.

        Parameters:
        - matrix (numpy.ndarray): The matrix to be vectorized.
        - include_diagonal (bool, optional): Flag to include diagonal elements in the vectorization.
          Defaults to False.

        Returns:
        - numpy.ndarray: The vectorized form of the matrix.
        """
        # Determine the size of the matrix based on its first dimension
        matrix_size = matrix.shape[0]

        # Initialize an empty list to accumulate vector elements
        vector_elements = []

        # Iterate over columns and then rows to collect the relevant elements
        for col in range(matrix_size):
            for row in range(matrix_size):
                # Skip diagonal elements if not including them
                if row != col:
                    if row < col:
                        # Collect upper triangle elements
                        vector_elements.append(matrix[row, col])
                    elif include_diagonal and row == col + 1:
                        # Optionally include the diagonal elements immediately below the diagonal
                        vector_elements.append(matrix[row, col])

        return np.array(vector_elements)

    @staticmethod
    def anti_vectorize(vector, matrix_size, include_diagonal=False):
        """
        Reconstructs a matrix from its vector form, filling it vertically.

        The method fills the matrix by reflecting vector elements into the upper triangle
        and optionally including the diagonal elements based on the include_diagonal flag.

        Parameters:
        - vector (numpy.ndarray): The vector to be transformed into a matrix.
        - matrix_size (int): The size of the square matrix to be reconstructed.
        - include_diagonal (bool, optional): Flag to include diagonal elements in the reconstruction.
          Defaults to False.

        Returns:
        - numpy.ndarray: The reconstructed square matrix.
        """
        # Initialize a square matrix of zeros with the specified size
        matrix = np.zeros((matrix_size, matrix_size))

        # Index to keep track of the current position in the vector
        vector_idx = 0

        # Fill the matrix by iterating over columns and then rows
        for col in range(matrix_size):
            for row in range(matrix_size):
                # Skip diagonal elements if not including them
                if row != col:
                    if row < col:
                        # Reflect vector elements into the upper triangle and its mirror in the lower triangle
                        matrix[row, col] = vector[vector_idx]
                        matrix[col, row] = vector[vector_idx]
                        vector_idx += 1
                    elif include_diagonal and row == col + 1:
                        # Optionally fill the diagonal elements after completing each column
                        matrix[row, col] = vector[vector_idx]
                        matrix[col, row] = vector[vector_idx]
                        vector_idx += 1

        return matrix


In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from scipy.stats import pearsonr
from scipy.spatial.distance import jensenshannon
from sklearn.metrics import mean_absolute_error
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
import community.community_louvain as community_louvain
import os
def calculate_centralities(adj_matrix):
    if adj_matrix.shape[0] != adj_matrix.shape[1]:
        raise ValueError(f"Adjacency matrix is not square: shape={adj_matrix.shape}")
    print(f"Processing adjacency matrix of shape: {adj_matrix.shape}")

    G = nx.from_numpy_array(adj_matrix)
    partition = community_louvain.best_partition(G)

    # Calculate the participation coefficient with the partition
    pc_dict = participation_coefficient(G, partition)

    # Calculate averages of centrality measures
    pr = nx.pagerank(G, alpha=0.9)
    ec = nx.eigenvector_centrality_numpy(G, max_iter=100)
    bc = nx.betweenness_centrality(G, normalized=True, endpoints=False)
    ns = np.array(list(nx.degree_centrality(G).values())) * (len(G.nodes()) - 1)
    acc = nx.average_clustering(G, weight=None)

    # Average participation coefficient
    pc_avg = np.mean(list(pc_dict.values()))

    return {
        'pr': np.mean(list(pr.values())),
        'ec': np.mean(list(ec.values())),
        'bc': np.mean(list(bc.values())),
        'ns': ns,
        'pc': pc_avg,
        'acc': acc
    }

def participation_coefficient(G, partition):
    # Initialize dictionary for participation coefficients
    pc_dict = {}

    # Calculate participation coefficient for each node
    for node in G.nodes():
        node_degree = G.degree(node)
        if node_degree == 0:
            pc_dict[node] = 0.0
        else:
            # Count within-module connections
            within_module_degree = sum(1 for neighbor in G[node] if partition[neighbor] == partition[node])
            # Calculate participation coefficient
            pc_dict[node] = 1 - (within_module_degree / node_degree) ** 2

    return pc_dict

def evaluate_all(true_hr_matrices, predicted_hr_matrices, output_path=path_eval_matrics):
    print(true_hr_matrices.shape)
    print(predicted_hr_matrices.shape)

    num_subjects = true_hr_matrices.shape[0]
    results = []

    for i in range(num_subjects):
        true_matrix = true_hr_matrices[i, :, :]
        pred_matrix = predicted_hr_matrices[i, :, :]

        print(f"Evaluating subject {i+1} with matrix shapes: true={true_matrix.shape}, pred={pred_matrix.shape}")

        if true_matrix.shape != pred_matrix.shape or true_matrix.shape[0] != true_matrix.shape[1]:
            print(f"Error: Matrix shape mismatch or not square for subject {i+1}: true={true_matrix.shape}, pred={pred_matrix.shape}")
            continue

        metrics = {
            'ID': i + 1,
            'MAE': mean_absolute_error(true_matrix.flatten(), pred_matrix.flatten()),
            'PCC': pearsonr(true_matrix.flatten(), pred_matrix.flatten())[0],
            'JSD': jensenshannon(true_matrix.flatten(), pred_matrix.flatten()),
        }

        true_metrics = calculate_centralities(true_matrix)
        pred_metrics = calculate_centralities(pred_matrix)

        for key in ['NS', 'PR', 'EC', 'BC', 'PC', 'ACC']:
            metrics[f'MAE in {key}'] = mean_absolute_error([true_metrics[key.lower()]], [pred_metrics[key.lower()]])

        results.append(metrics)

    df = pd.DataFrame(results)
    if not df.empty:
        # Check if the file exists to decide whether to write headers
        file_exists = os.path.isfile(output_path)

        df.to_csv(output_path, mode='a', header=not file_exists, index=False)
        print(f"Results appended to {output_path}.")
    else:
        print("No data to save.")



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
from scipy.spatial.distance import jensenshannon
import networkx as nx
import random, torch

# Set a fixed random seed for reproducibility across multiple libraries
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# Check for CUDA (GPU support) and set device accordingly
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # For multi-GPU setups
    # Additional settings for ensuring reproducibility on CUDA
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    device = torch.device("cpu")
    print("CUDA not available. Using CPU.")

def preprocess_data(data_path):
    # Load data
    data = pd.read_csv(data_path)

    # Data cleansing: replace negative and NaN values with 0
    data = np.maximum(data, 0)
    data = np.nan_to_num(data)

    return data

def vectorize(data):
    # Vectorization (if needed)
    vectorizer = MatrixVectorizer()
    vectors = vectorizer.vectorize(data)

    return vectors

def anti_vectorize(data, size, diagonal=False):
    # Reverse the vectorization (if needed)
    graphs = np.zeros((data.shape[0], size, size))
    for idx, graph in enumerate(data):
        vectorizer = MatrixVectorizer()
        graphs[idx] = vectorizer.anti_vectorize(graph, size, diagonal)

    return graphs

if option == 1:
    lr_data_path = path_lr_data
    hr_data_path = path_hr_data

    lr_data_path_test = path_lr_data_test
    hr_data_path_test = path_hr_data_test

if option == 0:
    lr_data_path = path_lr_data_tr
    hr_data_path = path_hr_data_tr

    lr_data_path_test = path_lr_data_valid
    hr_data_path_test = path_hr_data_valid

# lr_matrix: N * 12720, hr_matrix: N * 35778
lr_matrix = preprocess_data(lr_data_path)
hr_matrix = preprocess_data(hr_data_path)

lr_matrix_test = preprocess_data(lr_data_path_test)
hr_matrix_test = preprocess_data(hr_data_path_test)

# Define a function to calculate statistics and return them in a dictionary
def calculate_statistics(data):
    statistics = {
        'Mean': np.mean(data),
        'Median': np.median(data),
        'Standard Deviation': np.std(data),
        'Min': np.min(data),
        'Max': np.max(data)
    }
    return statistics

# Calculate statistics for LR and HR data
lr_stats = calculate_statistics(lr_matrix)
hr_stats = calculate_statistics(hr_matrix)

# Create a DataFrame to hold the statistics for comparison
df_stats = pd.DataFrame({'LR Data': lr_stats, 'HR Data': hr_stats})

# Round the numbers to four decimal places for better readability
df_stats = df_stats.round(4)

def plot_evaluation_metrics(fold_results):
    metrics = np.array(fold_results)

    # Calculate mean and standard deviation across folds for each metric
    metrics_mean = metrics.mean(axis=0)
    metrics_std = metrics.std(axis=0)

    # Define metric names
    metric_names = ['MAE', 'PCC', 'JSD', 'MAE-BC', 'MAE-EC', 'MAE-PR']

    # Determine the number of subplot rows and columns (up to 2 plots per line)
    n_folds = len(fold_results)
    n_rows = 2
    n_cols = (n_folds + 1) // n_rows + ((n_folds + 1) % n_rows > 0)

    fig, axs = plt.subplots(n_rows, n_cols, figsize=(20, 10), sharey=True)

    # Flatten the axs array for easy indexing
    axs = axs.flatten()

    # Plot each fold's metrics
    for i in range(n_folds):
        axs[i].bar(metric_names, metrics[i], color='skyblue')
        axs[i].set_title(f'Fold {i + 1}')

    # Adjust subplot index for the average metrics based on the number of folds
    avg_metrics_index = n_folds

    # Plot the average metrics with error bars on the next subplot
    axs[avg_metrics_index].bar(metric_names, metrics_mean, color='orange', yerr=metrics_std, capsize=5)
    axs[avg_metrics_index].set_title('Avg. Across Folds')

    # Adding error bars to the average metrics
    for i, (mean, std) in enumerate(zip(metrics_mean, metrics_std)):
        axs[avg_metrics_index].errorbar(metric_names[i], mean, yerr=std, fmt='k_', ecolor='black', capsize=5)

    plt.tight_layout()
    plt.savefig('evaluation_metrics.png', dpi=300, bbox_inches='tight')

def create_graph_from_vector(vectorized_matrix, matrix_size):
  adjacency_matrix = MatrixVectorizer.anti_vectorize(vectorized_matrix, matrix_size)
  G = nx.from_numpy_array(adjacency_matrix)
  return G

# N * 12720
X_tr = np.array(lr_matrix)
# N * 35778
Y_tr = np.array(hr_matrix)
#print(X.shape, y.shape)
X_test = np.array(lr_matrix_test)
# N * 35778
Y_test = np.array(hr_matrix_test)


#fold_results = []

X_train_tmp, X_test_tmp = X_tr, X_test
y_train_tmp, y_test = Y_tr, Y_test

# Initialize your GNN model here
model = AGRNet()
X_train = anti_vectorize(X_train_tmp, 160)
X_test = anti_vectorize(X_test_tmp, 160)
y_train = anti_vectorize(y_train_tmp, 268)
y_test_graph = anti_vectorize(y_test, 268)
# ################################################################################
# # Train final model using all data
# N * 12720
#print('Now training using all the data')
#X = np.array(lr_matrix)
# N * 35778
#y = np.array(hr_matrix)
#print(X.shape, y.shape)

model = AGRNet()
#X_train = anti_vectorize(X, 160)
#y_train = anti_vectorize(y, 268)

model.train(X_train, y_train, X_train, y_train)
################################################################################

predictions_tmp = model.predict(X_test)

metrics = evaluate_all(
    y_test_graph, predictions_tmp
    )
